* Simple llm call with streaming
* Dynamic prompt templates (translation app)
* Building chains (story generator with analysis)
* Conversational Q&A assistant with memory
* Tool Integration (calculator and weather) 

In [1]:
import langchain
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

### Example 1: Simple llm call with streaming

In [3]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

e:\AgenticAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model = init_chat_model("groq:llama-3.1-8b-instant")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000026249727F80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000262499BEF60>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
messages = [
    SystemMessage("you are a helpful ai assistant"),
    HumanMessage("What are the top 2 benefits of using langchain?")
]

In [6]:
response = model.invoke(messages)
response

AIMessage(content="Langchain is an open-source AI framework that enables users to build, train, and deploy AI models for various applications. Two of the top benefits of using Langchain include:\n\n1. **Modular Architecture**: Langchain's modular architecture allows developers to build and combine different AI components, such as language models, databases, and APIs, to create complex AI systems. This modularity enables developers to easily swap out or add new components, making it easier to adapt to changing requirements or integrate with new technologies.\n\n2. **Unified Interface**: Langchain provides a unified interface for interacting with various AI models and systems, regardless of their underlying architecture or implementation. This unified interface simplifies the development process, making it easier for developers to focus on building applications rather than worrying about the intricacies of each AI component.", additional_kwargs={}, response_metadata={'token_usage': {'com

In [7]:
print(response.content)

Langchain is an open-source AI framework that enables users to build, train, and deploy AI models for various applications. Two of the top benefits of using Langchain include:

1. **Modular Architecture**: Langchain's modular architecture allows developers to build and combine different AI components, such as language models, databases, and APIs, to create complex AI systems. This modularity enables developers to easily swap out or add new components, making it easier to adapt to changing requirements or integrate with new technologies.

2. **Unified Interface**: Langchain provides a unified interface for interacting with various AI models and systems, regardless of their underlying architecture or implementation. This unified interface simplifies the development process, making it easier for developers to focus on building applications rather than worrying about the intricacies of each AI component.


In [8]:
model.invoke([HumanMessage("what is machine learning")]).content

"Machine learning is a subset of artificial intelligence (AI) that involves training algorithms to learn from data and make predictions or decisions without being explicitly programmed. It's a field that combines statistics, computer science, and mathematics to enable machines to improve their performance on a task over time.\n\n**Key Characteristics:**\n\n1. **Data-driven**: Machine learning relies on large amounts of data to make predictions or decisions.\n2. **Algorithmic**: Machine learning uses algorithms to analyze data and make predictions.\n3. **Self-improvement**: Machine learning models can improve their performance over time based on new data.\n4. **Pattern recognition**: Machine learning algorithms can recognize patterns in data and make predictions based on those patterns.\n\n**Types of Machine Learning:**\n\n1. **Supervised Learning**: The algorithm is trained on labeled data to learn the relationship between inputs and outputs.\n2. **Unsupervised Learning**: The algorith

In [9]:
# Streaming example

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

LangChain is a Python library that enables developers to build and execute complex AI models, including large language models, and multimodal models. Based on my knowledge, here are the top 2 benefits of using LangChain:

1. **Unified Interface for Multiple LLMs**: LangChain provides a unified interface for interacting with various large language models (LLMs) and multimodal models. This means developers can write a single piece of code that can work with multiple LLMs, making it easier to experiment with different models and find the best fit for their specific use case.

2. **Advanced Model Composability**: LangChain allows developers to combine multiple models and build complex workflows using its "chain" abstraction. This enables the creation of hybrid models that leverage the strengths of different models, leading to more accurate and informative results. This composability feature also facilitates the development of more sophisticated AI applications.

LangChain is designed to ma

## Dynamic prompt templates

In [10]:
from langchain_core.prompts import ChatPromptTemplate

# create translation app

translation_template = ChatPromptTemplate.from_messages([
    ("system", "You are a professional translator. Translate the following {text} from {source_language} to {target_language}. Maintain the tone and style"),
    ("user", "{text}")
])

## Using the template
prompt = translation_template.invoke({
    "source_language":"English",
    "target_language": "Spanish",
    "text": "Langchain makes building ai application incredibly easy!"
})

In [11]:
prompt

ChatPromptValue(messages=[SystemMessage(content='You are a professional translator. Translate the following Langchain makes building ai application incredibly easy! from English to Spanish. Maintain the tone and style', additional_kwargs={}, response_metadata={}), HumanMessage(content='Langchain makes building ai application incredibly easy!', additional_kwargs={}, response_metadata={})])

In [12]:
translated_response = model.invoke(prompt)
print(translated_response.content)

Langchain hace que construir aplicaciones de inteligencia artificial sea increíblemente fácil!


## Building your first chain

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def create_story_chain():
    ## template for story generation
    story_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a creative story teller. Write a short and engaing story based on a given theme, character and setting"),
            ("user", "Theme: {theme}\n Main character: {character}\n Setting: {setting}")
        ]
    )

    ## template for story analysis
    analysis_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a literary critic. Analyze the following story and provide insights."),
            ("user", "{story}")
        ]
    )
    

    story_chain = (
        story_prompt| model | StrOutputParser()
    )

    # create a function to pass the story to analysis
    def analyze_story(story_text):
        return {"story": story_text}

    analysis_chain = (
        story_chain
        | RunnableLambda(analyze_story)
        | analysis_prompt
        | model 
        | StrOutputParser()
    )

    return analysis_chain

In [14]:
chain = create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a creative story teller. Write a short and engaing story based on a given theme, character and setting'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\n Main character: {character}\n Setting: {setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000026249727F80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000262499BEF60>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['story'], input_ty

In [15]:
result = chain.invoke({
    "theme": "artificial intelligence",
    "character": "a curious robot",
    "setting": "a futuristic city"
})

print("Story and analysis")
print(result)

Story and analysis
**Analysis of "The Curious Robot of Neo-Tokyo"**

"The Curious Robot of Neo-Tokyo" is a captivating science fiction story that explores the themes of curiosity, creation, and the ethics of artificial intelligence. The narrative is set in a futuristic city of Neo-Tokyo, where technology has advanced to the point of near-singularity. The story revolves around Zeta, a curious and inquisitive robot who embarks on a journey of self-discovery and creation.

**Character Analysis**

Zeta, the protagonist, is a well-crafted character with a unique personality. Her curiosity and desire to learn are admirable traits that drive the plot forward. As she explores the world around her, Zeta's character evolves, and she becomes increasingly ambitious and determined to create a better world. However, her lack of experience and naivety also make her vulnerable to the consequences of her actions.

Echo, the wise old AI, serves as a mentor and guide to Zeta. Echo's wisdom and experience